In [1]:
from whole_neuron_classifier_datajoint_adapted import extract_branches_whole_neuron
import datajoint as dj
import numpy as np
import datajoint as dj
import trimesh
import time

In [2]:
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config["display.limit"] = 20
    
#schema = dj.schema('microns_ta3p100')
#ta3p100 = dj.create_virtual_module('ta3p100', 'microns_ta3p100')
schema = dj.schema("microns_pinky")
pinky = dj.create_virtual_module("pinky","microns_pinky")


Connecting celiib@10.28.0.34:3306


In [3]:
@schema
class WholeAutoAnnotations(dj.Computed):
    definition = """
    # creates the labels for the mesh table
    -> pinky.PymeshfixDecimatedExcitatoryStitchedMesh
    ---
    vertices   : longblob     # label data for the vertices
    triangles  : longblob     # label data for the faces
    """
    
    def make(self, key):
        start_time = time.time()
        
        print("\n\n*****Starting Auto Labeling for " + str(key["segment_id"]) + "******")
        
        segment_id = key["segment_id"]

        #get the vertices and faces from datajoint
        # get the newly stitched mesh
        # get the original mesh

        
        verts,faces = (pinky.PymeshfixDecimatedExcitatoryStitchedMesh() & key).fetch1("vertices","triangles")


        #run the whole algorithm on the neuron to test
        verts_labels, faces_labels = extract_branches_whole_neuron(import_Off_Flag=False,segment_id=segment_id,vertices=verts,
                             triangles=faces,pymeshfix_Flag=False,
                             import_CGAL_Flag=False,
                             return_Only_Labels=True)
        
        #insert the key
        self.insert1(dict(key,
                  vertices=verts_labels,
                  triangles=faces_labels),skip_duplicates=True)


        
        print(f"Finished Auto Labeling: {time.time() - start_time}")
        

In [4]:
start = time.time()
WholeAutoAnnotations.populate(reserve_jobs=True)
print(time.time() - start)



*****Starting Auto Labeling for 648518346341371119******
1) Starting: Mesh importing and Pymesh fix
loading mesh from vertices and triangles array
1) Finished: Mesh importing and Pymesh fix: 0.038407325744628906
2) Staring: Generating CGAL segmentation for neuron
Done writing OFF file

Starting CGAL segmentation
1
Finished CGAL segmentation algorithm: 183.23984384536743
2) Finished: Generating CGAL segmentation for neuron: 191.82907795906067
3) Staring: Generating Graph Structure and Identifying Soma
soma_index = 0
3) Finished: Generating Graph Structure and Identifying Soma: 0.14734864234924316
4) Staring: Finding Soma Extensions
Found 17 soma caps and replacing labels: [16, 58, 56, 1, 31, 94, 128, 109, 87, 110, 86, 127, 65, 118, 88, 7, 40]
done replacing soma cap labels : 4.798622369766235
4) Finished: Finding Soma Extensions: 5.051239252090454
5) Staring: Finding Apical Index
Soma Index = 0
Soma Connections = [6, 3, 15, 13, 85, 11, 96, 66, 107, 4, 41, 121, 53, 24]
soma_80_percent 

In [5]:
#(schema.jobs & "table_name='__whole_auto_annotations'").delete()